In [11]:
from pathlib import Path
import importlib, resume_parser as rp
importlib.reload(rp)
import time

PRED_PATH = Path.cwd() / "preds.jsonl"        # e.g., .../notebooks/preds.jsonl
PRED_PATH.parent.mkdir(parents=True, exist_ok=True)

# where your sample resumes live
SAMPLES = Path("./samples")   # or Path.cwd() / "samples"
assert SAMPLES.exists(), f"Missing folder: {SAMPLES}"
assert any(p.suffix.lower() in {".pdf",".docx"} for p in SAMPLES.iterdir()), "No PDF/DOCX in samples"

MODEL = "llama3.1:8b" 
# where to save predictions
PRED_PATH = (Path.cwd() / "pred /preds.jsonl").resolve()
PRED_PATH.parent.mkdir(parents=True, exist_ok=True)
print("Samples:", SAMPLES.resolve())
print("Will write:", PRED_PATH)


Samples: C:\Users\parri\OneDrive\Documents\Parita\projects\resume_parser\samples
Will write: C:\Users\parri\OneDrive\Documents\Parita\projects\resume_parser\pred\preds.jsonl


In [3]:
with open(PRED_PATH, "w", encoding="utf-8") as f:
    for p in sorted(SAMPLES.iterdir()):
        if p.suffix.lower() not in {".pdf", ".docx"}:
            continue
        t0 = time.time()
        out = rp.parse_resume_llm(p, model=MODEL)  # llama3.1:8b only
        rec = out.model_dump()
        rec["file"] = p.name
        rec["latency_s"] = round(time.time() - t0, 2)
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")